In [1]:
!pip install datatable

In [1]:
import datatable as dt

In [2]:
data = dt.fread("../data/transactions_train.csv")

print("Train size:", data.shape)


Train size: (31788324, 5)


In [3]:
print(data[0, :])
print(data[31788323, :])

   | t_dat       customer_id                                                       article_id      price  sales_channel_id
   | date32      str32                                                                  int32    float64             int32
-- + ----------  ----------------------------------------------------------------  ----------  ---------  ----------------
 0 | 2018-09-20  000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318   663713001  0.0508305                 2
[1 row x 5 columns]

   | t_dat       customer_id                                                       article_id      price  sales_channel_id
   | date32      str32                                                                  int32    float64             int32
-- + ----------  ----------------------------------------------------------------  ----------  ---------  ----------------
 0 | 2020-09-22  fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20e02ce5d1e58a8f700b   898573003  0.0338814              

In [4]:
frequency = data[:, dt.count(), dt.by(dt.f.article_id)]

# Printing the head of the frequency frame
print(frequency.head())

   | article_id  count
   |      int32  int64
-- + ----------  -----
 0 |  108775015  10841
 1 |  108775044   7250
 2 |  108775051    215
 3 |  110065001   1044
 4 |  110065002    539
 5 |  110065011   1090
 6 |  111565001   4911
 7 |  111565003     46
 8 |  111586001  14191
 9 |  111593001  13888
[10 rows x 2 columns]



In [5]:
frequency = frequency[:, :, dt.sort(-dt.f.count)]

# Select the top 'n' rows
n = 10  # Replace with the number of top elements you want
top_n_frequency = frequency[:n, :]

# Print the result
print(top_n_frequency)

   | article_id  count
   |      int32  int64
-- + ----------  -----
 0 |  706016001  50287
 1 |  706016002  35043
 2 |  372860001  31718
 3 |  610776002  30199
 4 |  759871002  26329
 5 |  464297007  25025
 6 |  372860002  24458
 7 |  610776001  22451
 8 |  399223001  22236
 9 |  706016003  21241
[10 rows x 2 columns]



In [6]:
df = data[:, :-2]

In [7]:
print(df.head())

   | t_dat       customer_id                                                       article_id
   | date32      str32                                                                  int32
-- + ----------  ----------------------------------------------------------------  ----------
 0 | 2018-09-20  000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318   663713001
 1 | 2018-09-20  000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318   541518023
 2 | 2018-09-20  00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2   505221004
 3 | 2018-09-20  00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2   685687003
 4 | 2018-09-20  00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2   685687004
 5 | 2018-09-20  00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2   685687001
 6 | 2018-09-20  00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2   505221001
 7 | 2018-09-20  00083cda041544b2fbb0e0d2905ad17da7cf1007526

In [8]:
from datetime import datetime, timedelta

# Assuming 'df' is your datatable Frame with 't_dat' in date32 format and 'customer_id'

# Convert 't_dat' (date32 format) to week numbers
base_date = datetime(1970, 1, 1).date()
df['week_number'] = dt.Frame([(base_date + timedelta(days=int(day))).isocalendar()[1] 
                              for day in df['t_dat'].to_list()[0]])

# Group by customer_id and week_number, then count the purchases
weekly_purchases = df[:, dt.count(), by('customer_id', 'week_number')]

# Calculate the average number of purchases per week for each customer
average_purchases = weekly_purchases[:, dt.mean(dt.f.count), by('customer_id')]

# Print the head of the result
print(average_purchases.head())

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'datetime.date'

In [9]:
# Assuming 'df' is your datatable Frame with 't_dat' as datetime.date objects and 'customer_id'

# Convert 't_dat' to a combined year-week identifier
df['year_week'] = dt.Frame([f"{date.year}-{date.isocalendar()[1]:02d}" for date in df['t_dat'].to_list()[0]])

# Group by customer_id and year_week, then count the purchases
weekly_purchases = df[:, dt.count(), dt.by('customer_id', 'year_week')]

# Calculate the average number of purchases per week for each customer
average_purchases = weekly_purchases[:, dt.mean(dt.f.count), dt.by('customer_id')]

# Print the head of the result
print(average_purchases.head())


   | customer_id                                                          count
   | str32                                                              float64
-- + ----------------------------------------------------------------  --------
 0 | 00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657   2.1    
 1 | 0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa   4.52632
 2 | 000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318   2.57143
 3 | 00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2c5feb1ca5dff07c43e   2      
 4 | 00006413d8573cd20ed7128e53b7b13819fe5cfc2d801fe7fc0f26dd8d65a85a   2.16667
 5 | 000064249685c11552da43ef22a5030f35a147f723d5b02ddd9fd22452b1f5a6   3      
 6 | 0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d8cd0c725276a467a2a   2      
 7 | 00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2  10      
 8 | 00007e8d4e54114b5b2a9b51586325a8d0fa74ea23ef77334eaec4ffccd7ebcc   2      
 9 | 00008469a21b50b3d147c97135e25b4201a

In [10]:
average_purchases.to_csv("avg_customer_purchases.csv")

In [11]:
data = dt.fread("avg_customer_purchases.csv")

print("Train size:", data.shape)


Train size: (1362281, 2)


In [12]:
print(data[0,:])

   | customer_id                                                         count
   | str32                                                             float64
-- + ----------------------------------------------------------------  -------
 0 | 00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657      2.1
[1 row x 2 columns]

